In [2]:
import requests
import pandas as pd
import boto3
from datetime import datetime
from io import BytesIO

# Configuration
S3_BUCKET = "lab2bucket-lnu"

def s3_upload_file_in_memory(file_buffer, bucket_name, file_name):
    """Upload file buffer to S3."""
    s3_client = boto3.client('s3')
    try:
        s3_client.upload_fileobj(file_buffer, bucket_name, file_name)
        print(f"Файл {file_name} завантажено в S3 бакет {bucket_name}.")
    except Exception as e:
        print(f"Помилка завантаження {file_name}: {e}")

def fetch_exchange_rates(start_year, start_month=1, end_month=12, bucket_name=S3_BUCKET):
    """Fetch exchange rates for a given year and upload to S3."""
    if not (1 <= start_month <= 12) or not (1 <= end_month <= 12):
        print("Задайте місяць цифрою від 1 до 12.")
        return
    
    # Form start and end dates in YYYYMMDD format
    start_date = f"{start_year}{str(start_month).zfill(2)}01"
    end_day = pd.Timestamp(year=start_year, month=end_month, day=1).days_in_month
    end_date = f"{start_year}{str(end_month).zfill(2)}{str(end_day).zfill(2)}"
    
    # API URLs for USD and EUR
    url_usd = f"https://bank.gov.ua/NBU_Exchange/exchange_site?start={start_date}&end={end_date}&valcode=USD&sort=exchangedate&json"
    url_eur = f"https://bank.gov.ua/NBU_Exchange/exchange_site?start={start_date}&end={end_date}&valcode=EUR&sort=exchangedate&json"
    
    # Fetch data
    try:
        response_usd = requests.get(url_usd)
        response_eur = requests.get(url_eur)
        if response_usd.status_code == 200 and response_eur.status_code == 200:
            data_usd = response_usd.json()
            data_eur = response_eur.json()
        else:
            print("Помилка отримання даних від API")
            return
    except Exception as e:
        print(f"Помилка запиту до API: {e}")
        return
    
    # Convert to DataFrames
    df_usd = pd.DataFrame(data_usd)
    df_eur = pd.DataFrame(data_eur)
    
    # Merge DataFrames on exchangedate
    if not df_usd.empty and not df_eur.empty:
        df_combined = pd.merge(
            df_usd[['exchangedate', 'rate']],
            df_eur[['exchangedate', 'rate']],
            on='exchangedate',
            suffixes=('_USD', '_EUR')
        )
    else:
        print("Дані для одного з валют відсутні.")
        return
    
    # Save to CSV in memory
    file_name = f"exchange_rate_{start_year}_{str(start_month).zfill(2)}_{str(end_month).zfill(2)}.csv"
    csv_buffer = BytesIO()
    df_combined.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)
    
    # Upload to S3
    s3_upload_file_in_memory(csv_buffer, bucket_name, f"exchange_rates/{file_name}")
    csv_buffer.close()

# Fetch data for 2022 (January to December)
fetch_exchange_rates(2022, 1, 12)

Файл exchange_rates/exchange_rate_2022_01_12.csv завантажено в S3 бакет lab2bucket-lnu.


In [4]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO, BytesIO

# Configuration
S3_BUCKET = "lab2bucket-lnu"
FILE_NAME = "exchange_rates/exchange_rate_2022_01_12.csv"
PLOT_FILENAME = "uah_exchange_rates_2022.png"
DATA_OUTPUT_FILE = "exchange_rate_data_2022.csv"

def s3_download_file(bucket_name, file_name):
    """Download file from S3 into memory."""
    s3_client = boto3.client("s3")
    try:
        obj = s3_client.get_object(Bucket=bucket_name, Key=file_name)
        file_content = obj['Body'].read().decode('utf-8')
        return file_content
    except Exception as e:
        print(f"Error downloading file {file_name}: {e}")
        return None

def s3_upload_file_in_memory(file_buffer, bucket_name, file_name):
    """Upload file buffer to S3."""
    s3_client = boto3.client('s3')
    try:
        s3_client.upload_fileobj(file_buffer, bucket_name, file_name)
        print(f"Файл {file_name} завантажено в S3 бакет {bucket_name}.")
    except Exception as e:
        print(f"Помилка завантаження {file_name}: {e}")

def plot_exchange_rates(file_name=FILE_NAME, bucket_name=S3_BUCKET):
    """Plot exchange rates from S3 data, output UI data, and upload the plot."""
    # Download and read CSV
    csv_data = s3_download_file(bucket_name, file_name)
    if csv_data is None:
        return
    
    try:
        df = pd.read_csv(StringIO(csv_data))
    except Exception as e:
        print(f"Error reading CSV data: {e}")
        return
    
    # Convert exchangedate to datetime
    try:
        df['exchangedate'] = pd.to_datetime(df['exchangedate'], format='%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing dates: {e}")
        print("Sample dates in 'exchangedate' column:")
        print(df['exchangedate'].head())
        return
    
    # Format the data to match the plot's UI (daily exchange rates for USD and EUR)
    # Rename columns to match the plot's labels in Ukrainian
    ui_df = df.rename(columns={
        'exchangedate': 'Дата',
        'rate_USD': 'Курс USD (UAH)',
        'rate_EUR': 'Курс EUR (UAH)'
    })
    
    # Print the UI data as a table
    print("\nДані для графіка (Курс гривні до USD та EUR за 2022 рік):")
    print(ui_df.to_string(index=False, float_format="%.4f"))
    
    # Save the UI data to a new CSV file in memory
    csv_buffer = BytesIO()
    ui_df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)
    
    # Upload the UI data CSV to S3
    s3_upload_file_in_memory(csv_buffer, bucket_name, f"exchange_rates/{DATA_OUTPUT_FILE}")
    csv_buffer.close()
    
    # Set plot style
    plt.style.use('grayscale')
    plt.figure(figsize=(12, 6))
    plt.gcf().set_facecolor('#BEBEBE')
    
    # Plot USD and EUR
    plt.plot(df['exchangedate'], df['rate_USD'], label='$ USD', color='green')
    plt.plot(df['exchangedate'], df['rate_EUR'], label='€ EUR', color='blue')
    
    # Customize plot to match the provided image
    plt.title('Курс гривні до USD та EUR за 2022 рік', fontsize=16, weight='bold')
    plt.xlabel('Дата', fontsize=14)
    plt.ylabel('Курс (UAH)', fontsize=14)
    plt.legend(title='Валюта', fontsize=12, title_fontsize='13', frameon=True, edgecolor='black')
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(range(int(df['rate_USD'].min()), int(df['rate_EUR'].max()) + 1, 1))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    
    # Save plot to buffer
    plot_buffer = BytesIO()
    plt.savefig(plot_buffer, format='png')
    plot_buffer.seek(0)
    
    # Upload plot to S3
    s3_upload_file_in_memory(plot_buffer, bucket_name, f"plots/{PLOT_FILENAME}")
    
    # Close resources
    plt.close()
    plot_buffer.close()

# Run the plotting function
plot_exchange_rates()


Дані для графіка (Курс гривні до USD та EUR за 2022 рік):
      Дата  Курс USD (UAH)  Курс EUR (UAH)
2022-01-01         27.2782         30.9226
2022-01-02         27.2782         30.9226
2022-01-03         27.2782         30.9226
2022-01-04         27.2782         30.9226
2022-01-05         27.2976         30.7739
2022-01-06         27.4551         31.0421
2022-01-07         27.4551         31.0421
2022-01-08         27.4551         31.0421
2022-01-09         27.4551         31.0421
2022-01-10         27.4977         31.1095
2022-01-11         27.4998         31.1160
2022-01-12         27.5093         31.1722
2022-01-13         27.7073         31.4796
2022-01-14         27.7372         31.7813
2022-01-15         27.7372         31.7813
2022-01-16         27.7372         31.7813
2022-01-17         27.9514         31.9890
2022-01-18         28.1220         32.0577
2022-01-19         28.4101         32.3406
2022-01-20         28.4038         32.1943
2022-01-21         28.3236         32.